
# Visualizing the stock market structure

This example employs several unsupervised learning techniques to extract
the stock market structure from variations in historical quotes.

The quantity that we use is the daily variation in quote price: quotes
that are linked tend to fluctuate in relation to each other during a day.


In [11]:
# Author: Gael Varoquaux gael.varoquaux@normalesup.org
# License: BSD 3 clause

## Retrieve the data from Internet

The data is from 2003 - 2008. This is reasonably calm: (not too long ago so
that we get high-tech firms, and before the 2008 crash). This kind of
historical data can be obtained from APIs like the
[data.nasdaq.com](https://data.nasdaq.com/) and
[alphavantage.co](https://www.alphavantage.co/).



In [12]:
import sys

import numpy as np
import pandas as pd

symbol_dict = {
    "TOT": "Total",
    "XOM": "Exxon",
    "CVX": "Chevron",
    "COP": "ConocoPhillips",
    "VLO": "Valero Energy",
    "MSFT": "Microsoft",
    "IBM": "IBM",
    "TWX": "Time Warner",
    "CMCSA": "Comcast",
    "CVC": "Cablevision",
    "YHOO": "Yahoo",
    "DELL": "Dell",
    "HPQ": "HP",
    "AMZN": "Amazon",
    "TM": "Toyota",
    "CAJ": "Canon",
    "SNE": "Sony",
    "F": "Ford",
    "HMC": "Honda",
    "NAV": "Navistar",
    "NOC": "Northrop Grumman",
    "BA": "Boeing",
    "KO": "Coca Cola",
    "MMM": "3M",
    "MCD": "McDonald's",
    "PEP": "Pepsi",
    "K": "Kellogg",
    "UN": "Unilever",
    "MAR": "Marriott",
    "PG": "Procter Gamble",
    "CL": "Colgate-Palmolive",
    "GE": "General Electrics",
    "WFC": "Wells Fargo",
    "JPM": "JPMorgan Chase",
    "AIG": "AIG",
    "AXP": "American express",
    "BAC": "Bank of America",
    "GS": "Goldman Sachs",
    "AAPL": "Apple",
    "SAP": "SAP",
    "CSCO": "Cisco",
    "TXN": "Texas Instruments",
    "XRX": "Xerox",
    "WMT": "Wal-Mart",
    "HD": "Home Depot",
    "GSK": "GlaxoSmithKline",
    "PFE": "Pfizer",
    "SNY": "Sanofi-Aventis",
    "NVS": "Novartis",
    "KMB": "Kimberly-Clark",
    "R": "Ryder",
    "GD": "General Dynamics",
    "RTN": "Raytheon",
    "CVS": "CVS",
    "CAT": "Caterpillar",
    "DD": "DuPont de Nemours",
}


symbols, names = np.array(sorted(symbol_dict.items())).T

quotes = []

for symbol in symbols:
    print("Fetching quote history for %r" % symbol, file=sys.stderr)
    url = (
        "https://raw.githubusercontent.com/scikit-learn/examples-data/"
        "master/financial-data/{}.csv"
    )
    quotes.append(pd.read_csv(url.format(symbol)))

close_prices = np.vstack([q["close"] for q in quotes])
open_prices = np.vstack([q["open"] for q in quotes])

# The daily variations of the quotes are what carry the most information
variation = close_prices - open_prices

Fetching quote history for 'AAPL'
Fetching quote history for 'AIG'
Fetching quote history for 'AMZN'
Fetching quote history for 'AXP'
Fetching quote history for 'BA'
Fetching quote history for 'BAC'
Fetching quote history for 'CAJ'
Fetching quote history for 'CAT'
Fetching quote history for 'CL'
Fetching quote history for 'CMCSA'
Fetching quote history for 'COP'
Fetching quote history for 'CSCO'
Fetching quote history for 'CVC'
Fetching quote history for 'CVS'
Fetching quote history for 'CVX'
Fetching quote history for 'DD'
Fetching quote history for 'DELL'
Fetching quote history for 'F'
Fetching quote history for 'GD'
Fetching quote history for 'GE'
Fetching quote history for 'GS'
Fetching quote history for 'GSK'
Fetching quote history for 'HD'
Fetching quote history for 'HMC'
Fetching quote history for 'HPQ'
Fetching quote history for 'IBM'
Fetching quote history for 'JPM'
Fetching quote history for 'K'
Fetching quote history for 'KMB'
Fetching quote history for 'KO'
Fetching quote h


## Learning a graph structure

We use sparse inverse covariance estimation to find which quotes are
correlated conditionally on the others. Specifically, sparse inverse
covariance gives us a graph, that is a list of connections. For each
symbol, the symbols that it is connected to are those useful to explain
its fluctuations.



In [13]:
from sklearn import covariance

alphas = np.logspace(-1.5, 1, num=10)
edge_model = covariance.GraphicalLassoCV(alphas=alphas)

# standardize the time series: using correlations rather than covariance
# former is more efficient for structure recovery
X = variation.copy().T
X /= X.std(axis=0)
edge_model.fit(X)

GraphicalLassoCV(alphas=array([ 0.03162278,  0.05994843,  0.11364637,  0.21544347,  0.40842387,
        0.77426368,  1.46779927,  2.7825594 ,  5.27499706, 10.        ]))

## Clustering using affinity propagation

We use clustering to group together quotes that behave similarly. Here,
amongst the `various clustering techniques <clustering>` available
in the scikit-learn, we use `affinity_propagation` as it does
not enforce equal-size clusters, and it can choose automatically the
number of clusters from the data.

Note that this gives us a different indication than the graph, as the
graph reflects conditional relations between variables, while the
clustering reflects marginal properties: variables clustered together can
be considered as having a similar impact at the level of the full stock
market.



In [14]:
from sklearn import cluster

_, labels = cluster.affinity_propagation(edge_model.covariance_, random_state=0)
n_labels = labels.max()

for i in range(n_labels + 1):
    print(f"Cluster {i + 1}: {', '.join(names[labels == i])}")

Cluster 1: Apple, Amazon, Yahoo
Cluster 2: Comcast, Cablevision, Time Warner
Cluster 3: ConocoPhillips, Chevron, Total, Valero Energy, Exxon
Cluster 4: Cisco, Dell, HP, IBM, Microsoft, SAP, Texas Instruments
Cluster 5: Boeing, General Dynamics, Northrop Grumman, Raytheon
Cluster 6: AIG, American express, Bank of America, Caterpillar, CVS, DuPont de Nemours, Ford, General Electrics, Goldman Sachs, Home Depot, JPMorgan Chase, Marriott, McDonald's, 3M, Ryder, Wells Fargo, Wal-Mart
Cluster 7: GlaxoSmithKline, Novartis, Pfizer, Sanofi-Aventis, Unilever
Cluster 8: Kellogg, Coca Cola, Pepsi
Cluster 9: Colgate-Palmolive, Kimberly-Clark, Procter Gamble
Cluster 10: Canon, Honda, Navistar, Sony, Toyota, Xerox


## Embedding in 2D space

For visualization purposes, we need to lay out the different symbols on a
2D canvas. For this we use `manifold` techniques to retrieve 2D
embedding.
We use a dense eigen_solver to achieve reproducibility (arpack is initiated
with the random vectors that we don't control). In addition, we use a large
number of neighbors to capture the large-scale structure.



In [15]:
# Finding a low-dimension embedding for visualization: find the best position of
# the nodes (the stocks) on a 2D plane

from sklearn import manifold

node_position_model = manifold.LocallyLinearEmbedding(
    n_components=3, eigen_solver="dense", n_neighbors=6
)

embedding = node_position_model.fit_transform(X.T).T

In [24]:
import plotly.graph_objects as go
import numpy as np

def create_3d_scatterplot(embedding):
    X = embedding[0]
    Y = embedding[1]
    Z = embedding[2]
    
    # Create the 3D scatterplot
    fig = go.Figure(data=[go.Scatter3d(x=X, y=Y, z=Z, mode='markers')])
    
    # Set plot layout (optional)
    fig.update_layout(scene=dict(
        xaxis_title='X Axis Title',
        yaxis_title='Y Axis Title',
        zaxis_title='Z Axis Title'
    ))
    
    # Show the plot (you can also save it to a file)
    fig.show()

# Assuming you have the 'embedding' data as a 3D array or list of lists
embedding = embedding  # Replace this with your actual data
create_3d_scatterplot(embedding)


In [1]:
import pandas as pd
import numpy as np
import numpy as np
import streamlit as st
from graph import create_3d_scatterplot
from sklearn.preprocessing import RobustScaler

add_selectbox = st.sidebar.selectbox(
    "How would you like to be contacted?", ("Home phone", "Beta Calculation", "Mobile phone")
)

# Load Data
df = pd.read_csv("Data/info_merged.csv", header=0, index_col=0)
df.reset_index(inplace=True)
df = df[df["currency"].isin(["EUR"])]
sector_selection = list(df["sector"].unique())
industry_selection = list(df["industry"].unique())

# Marketcap Bins
num = 10
labels = np.linspace(start=1, stop=num, num=num)
df["marketCap_bins"] = pd.qcut(df["marketCap"], q=num, labels=labels)

# Filter Data for TSNE
X = df[["marketCap_bins", "revenueGrowth", "operatingMargins"]]
X = RobustScaler().fit_transform(X)

fig = create_3d_scatterplot(df=df, X=X)

feature_select = st.multiselect("Features", df.columns)
sector_select = st.multiselect("Sector", sector_selection, ["Technology"])
industry_select = st.multiselect("Industry", industry_selection)

df = df[(df["sector"].isin(sector_select)) | (df["industry"].isin(industry_select))]
st.plotly_chart(fig, use_container_width=True)
st.dataframe(df, hide_index=True)


2023-09-26 14:39:42.369 
  command:

    streamlit run /home/chris/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [2]:
feature_select

[]